In [6]:
from vsql.relational_dbs.sqlite import VectorVirtualTableModule
from vsql.vector_dbs.faiss import FlatFaissVDB
from vsql.embedders.st import embed as st_embed
import apsw
from tabulate import tabulate

### Setup a dummy table and a vector index on it

In [2]:
# connect to the relational database using APSW (you can use this cursor just like a normal sqlite3 cursor)
conn = apsw.Connection(":memory:")
# declare a normal table
conn.execute("create table foo (id integer, name text, price integer)")
# register the vector database extension w/ sqlite 
index = FlatFaissVDB(ndim = 384)
conn.create_module("vdb_faiss_st", VectorVirtualTableModule(embedder = st_embed, index = index))
# declare a virtual table for vector search
conn.execute("create virtual table foo_vdb using vdb_faiss_st ()")
# use triggers to sync the tables
conn.execute("CREATE TRIGGER foo_ai AFTER INSERT ON foo BEGIN INSERT INTO foo_vdb (id, query, similarity) VALUES (new.id, new.name, 0); END;")
pass

In [3]:
products = [
    (1, 'Eco-Friendly Bamboo Desk Organizer', 35),
    (2, 'Handcrafted Ceramic Vase', 40),
    (3, 'Smart LED Light Strip', 60),
    (4, 'Portable Bluetooth Speaker', 50),
    (5, 'Aromatherapy Essential Oil Diffuser', 25),
    (6, 'Vintage Leather Journal', 30),
    (7, 'Gourmet Coffee Bean Sampler', 45),
    (8, 'Waterproof Outdoor Blanket', 20),
    (9, 'Adjustable Dumbbells Set', 80),
    (10, 'Digital Kitchen Scale', 15)
]
conn.executemany("insert into foo (id, name, price) values (?, ?, ?)", products)

### Query the tables

In [13]:
query = 'cooking'
recs = conn.execute("""
    SELECT name, price, similarity
    FROM foo_vdb JOIN foo ON foo.id=foo_vdb.id
    WHERE foo_vdb.query=? ORDER BY similarity
    LIMIT 3""",
(query,)).fetchall()
# pretty print the results
print("Top products similar to '{}'".format(query))
print(tabulate(recs, headers=["Product Name", "Price", "Similarity"], tablefmt="pretty", showindex=False))


Top products similar to 'cooking'
+-----------------------------+-------+--------------------+
|        Product Name         | Price |     Similarity     |
+-----------------------------+-------+--------------------+
|    Digital Kitchen Scale    |  15   | 1.5366830825805664 |
|  Handcrafted Ceramic Vase   |  40   | 1.6337971687316895 |
| Gourmet Coffee Bean Sampler |  45   | 1.802882432937622  |
+-----------------------------+-------+--------------------+
